In [ ]:
# Import necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from ant_colony import AntColony
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

# Load dataset
data = pd.read_csv(r"C:\Users\amnsh\OneDrive\Desktop\DAA\Dataset_cia2\Bank_Personal_Loan_Modelling.csv")

# Split dataset into input and output variables
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale input data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define neural network architecture
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    return model

# Define function to train neural network using ACO
def train_model(params):
    model = create_model(X_train.shape[1])
    model.set_weights(params)
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0, validation_split=0.2)
    y_pred = model.predict_classes(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return 1 - accuracy

# Define bounds for ACO parameters
bounds = [(-1, 1)] * sum([coef.size for coef in create_model(X_train.shape[1]).get_weights()])

# Train neural network using ACO
aco = AntColony(bounds, train_model, colony_size=10, max_iter=10, alpha=1, beta=2, evaporation_rate=0.5)
best_params, best_cost = aco.run()

# Create final model with best weights
final_model = create_model(X_train.shape[1])
final_model.set_weights(best_params)

# Evaluate final model on testing set
y_pred = final_model.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))
